# Import libraries

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels import api as sm
from sklearn.feature_selection import RFE
import scipy.stats as stats
from statsmodels.stats import weightstats
import statsmodels.stats.oneway as oneway
from scipy.stats import f_oneway
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

# Load data

In [2]:
df1 = pd.read_csv('Producer Prices.csv')
df2 = pd.read_csv('Consumer Price Indices.csv')
df3 = pd.read_csv('Temperature_Change.csv')
df4 = pd.read_csv('Value_Added_Agri_Frs_Fishing.csv')
ire_df1 = df1.loc[df1['Area'] == 'Ireland']  # based on df1 ('Producer Prices.csv')

# Data Understanding
#####  - Collecting and preparing data for analysis
#####  - Exploring and visualizing data
#####  - Verifying data quality


In [8]:
df1.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
0,PP,Producer Prices,40,Austria,5532,Producer Price (USD/tonne),1341.0,Apples,1991,1991,7021,Annual value,USD,823.9,A,Official figure
1,PP,Producer Prices,40,Austria,5532,Producer Price (USD/tonne),1341.0,Apples,1992,1992,7021,Annual value,USD,765.3,A,Official figure
2,PP,Producer Prices,40,Austria,5532,Producer Price (USD/tonne),1341.0,Apples,1993,1993,7021,Annual value,USD,564.0,A,Official figure
3,PP,Producer Prices,40,Austria,5532,Producer Price (USD/tonne),1341.0,Apples,1994,1994,7021,Annual value,USD,478.0,A,Official figure
4,PP,Producer Prices,40,Austria,5532,Producer Price (USD/tonne),1341.0,Apples,1995,1995,7021,Annual value,USD,486.0,A,Official figure


In [9]:
df1.tail()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
36835,PP,Producer Prices,752,Sweden,5532,Producer Price (USD/tonne),111.0,Wheat,2016,2016,7021,Annual value,USD,150.1,A,Official figure
36836,PP,Producer Prices,752,Sweden,5532,Producer Price (USD/tonne),111.0,Wheat,2017,2017,7021,Annual value,USD,151.1,A,Official figure
36837,PP,Producer Prices,752,Sweden,5532,Producer Price (USD/tonne),111.0,Wheat,2018,2018,7021,Annual value,USD,213.3,A,Official figure
36838,PP,Producer Prices,752,Sweden,5532,Producer Price (USD/tonne),111.0,Wheat,2019,2019,7021,Annual value,USD,157.4,A,Official figure
36839,PP,Producer Prices,752,Sweden,5532,Producer Price (USD/tonne),111.0,Wheat,2020,2020,7021,Annual value,USD,169.6,A,Official figure


In [10]:
df1.shape

(36840, 16)

In [11]:
df1.isna().sum()

Domain Code         0
Domain              0
Area Code (M49)     0
Area                0
Element Code        0
Element             0
Item Code (CPC)     0
Item                0
Year Code           0
Year                0
Months Code         0
Months              0
Unit                0
Value               0
Flag                0
Flag Description    0
dtype: int64

In [12]:
df1.duplicated().sum()

0

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36840 entries, 0 to 36839
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       36840 non-null  object 
 1   Domain            36840 non-null  object 
 2   Area Code (M49)   36840 non-null  int64  
 3   Area              36840 non-null  object 
 4   Element Code      36840 non-null  int64  
 5   Element           36840 non-null  object 
 6   Item Code (CPC)   36840 non-null  float64
 7   Item              36840 non-null  object 
 8   Year Code         36840 non-null  int64  
 9   Year              36840 non-null  int64  
 10  Months Code       36840 non-null  int64  
 11  Months            36840 non-null  object 
 12  Unit              36840 non-null  object 
 13  Value             36840 non-null  float64
 14  Flag              36840 non-null  object 
 15  Flag Description  36840 non-null  object 
dtypes: float64(2), int64(5), object(9)
memor

### Finding The 'Maximum' Values for Europe

In [18]:
# We use the "idxmax" function to find the maximum value in the Series for European Countries
df1['Value'].idxmax()

6325

In [21]:
"""
The 'Iloc' helps us find the maximum value in df1. In 2013, Cyprus's Maximum Producer Price was 29243.2 USD/tone, 
which was paid for "Game meat, fresh, chilled or frozen." 
"""
df1.iloc[6325-1]

Domain Code                                          PP
Domain                                  Producer Prices
Area Code (M49)                                     196
Area                                             Cyprus
Element Code                                       5532
Element                      Producer Price (USD/tonne)
Item Code (CPC)                                21170.02
Item                Game meat, fresh, chilled or frozen
Year Code                                          2013
Year                                               2013
Months Code                                        7021
Months                                     Annual value
Unit                                                USD
Value                                           29243.2
Flag                                                  A
Flag Description                        Official figure
Name: 6324, dtype: object

### Finding The Miminum Values for Europe

In [25]:
df1['Value'].idxmin()

29173

In [23]:
df1.iloc[29173-1]

Domain Code                                   PP
Domain                           Producer Prices
Area Code (M49)                              642
Area                                     Romania
Element Code                                5532
Element               Producer Price (USD/tonne)
Item Code (CPC)                           1229.0
Item                Cantaloupes and other melons
Year Code                                   1996
Year                                        1996
Months Code                                 7021
Months                              Annual value
Unit                                         USD
Value                                      143.1
Flag                                           A
Flag Description                 Official figure
Name: 29172, dtype: object

In [27]:
# As indicated in the 'Value' column, the minimum value in Europe shows information about "cantaloupes and other melons" 
# in Romania in 1996 and the producer price was 143.1 USD/tonne.

## Maximum Values for Republic of Ireland

In [28]:
# Additionally, we would like to find out what the minimum and maximum values are in Ireland
ire_df1['Value'].idxmax() 

19534

In [29]:
df1.iloc[19534-1]

Domain Code                                 PP
Domain                         Producer Prices
Area Code (M49)                            372
Area                                   Ireland
Element Code                              5532
Element             Producer Price (USD/tonne)
Item Code (CPC)                         1270.0
Item                    Mushrooms and truffles
Year Code                                 2007
Year                                      2007
Months Code                               7021
Months                            Annual value
Unit                                       USD
Value                                   3642.2
Flag                                         A
Flag Description               Official figure
Name: 19533, dtype: object

In [36]:
# The Mximum value in Ireland shows information about "Mushrooms and truffles" in 2007 and the 
# producer price was 3642.2USD/tonne.

## Minimum Values for Republic of Ireland

In [30]:
ire_df1['Value'].idxmin()

19627

In [31]:
df1.iloc[19627-1]

Domain Code                                 PP
Domain                         Producer Prices
Area Code (M49)                            372
Area                                   Ireland
Element Code                              5532
Element             Producer Price (USD/tonne)
Item Code (CPC)                         1801.0
Item                                Sugar beet
Year Code                                 1999
Year                                      1999
Months Code                               7021
Months                            Annual value
Unit                                       USD
Value                                     51.4
Flag                                         A
Flag Description               Official figure
Name: 19626, dtype: object

In [34]:
# The minimum value in Ireland shows information about "Sugar beet" in 1999 and the producer price was 51.4USD/tonne.

In [47]:
max_value = ire_df1['Value'].idxmax()
min_value = ire_df1['Value'].idxmin()
result = pd.DataFrame({'MaxValue': [max_value], 'MinValue': [min_value]})
result

,MaxValue,MinValue
0,19534,19627


## Data Cleaning